## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [2]:
pip install --upgrade nltk gensim bokeh scikit-learn

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ---------------------------------------- 24.0/24.0 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 6.8/6.8 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 1.5 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached scipy-1.11.4-cp310-cp310-win_amd64.whl (44.1 MB)
     ---------------------------------------- 57.0/57.0 kB 2.9 MB/s eta 0:00:00
  Using cached numpy-1.26.2-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached Pillow-10.1.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached pandas-2.1.4-cp310-cp310-win_amd64.whl (10.7 MB)
     ---------------------------------------- 56.3/56.3 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 186.7/186.7 kB 1.6 MB/s eta 0:00:00
  


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# download the data:
!curl https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    54    0    54    0     0    111      0 --:--:-- --:--:-- --:--:--   112
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: .


In [1]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [2]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(line.lower()) for line in data]

In [12]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [13]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [29]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=1).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-4.0345e-01,  2.7251e-01,  2.1853e-01,  3.2440e-01, -3.9078e-01,
        3.1568e-01,  6.7064e-01, -2.3172e-01,  2.7944e-01,  8.5353e-01,
        3.1557e-01, -7.1326e-01, -5.1827e+00, -1.5941e-01, -3.6804e-01,
        8.8010e-02, -4.8119e-01, -1.5729e-01,  2.3228e-02,  5.1899e-01,
        3.8920e-01,  1.6812e-01, -4.8545e-01,  3.0454e-01,  3.9643e-01,
        8.5730e-01,  3.2853e-01,  8.2905e-02,  8.4235e-02, -3.6721e-01,
        1.5191e-01,  4.5757e-01, -5.3428e-01,  3.5136e-01, -6.5984e-01,
        4.5150e-02,  3.7618e-01, -4.9815e-01, -2.4063e-01,  5.0031e-02,
       -4.3774e-01, -7.3420e-03,  7.1933e-02, -2.3923e-01,  5.2413e-01,
       -1.2460e-01,  1.3120e-01,  2.7869e-01, -5.6155e-01,  2.7182e-01,
       -2.4129e-01,  4.0371e-01, -8.9041e-02, -9.9831e-02,  4.4765e-01,
        5.2124e-01,  2.0899e-01, -4.1132e-02, -7.1021e-02,  3.6736e-01,
        2.3234e-01,  1.6121e-01,  2.1826e-01, -3.1339e-03,  2.9962e-01,
       -3.2263e-01, -2.8183e-01,  2.0653e-01, -4.0681e-01, -3.50

In [32]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9216071367263794),
 ('meat', 0.9087712168693542),
 ('milk', 0.89988112449646),
 ('grass', 0.8785324692726135),
 ('potatoes', 0.8779170513153076),
 ('fruit', 0.8755404949188232),
 ('garlic', 0.8715614676475525),
 ('fish', 0.8713198304176331),
 ('honey', 0.8654516339302063),
 ('pork', 0.865365207195282)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [3]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [9]:
model.most_similar(positive=["казань"], negative=["putin"])

[('ярославль', 0.6284331679344177),
 ('уфа', 0.6192706823348999),
 ('краснодар', 0.6143120527267456),
 ('самара', 0.6126881241798401),
 ('нижнекамск', 0.6064571142196655),
 ('красноярск', 0.606421947479248),
 ('воронеж', 0.6046156883239746),
 ('тюмень', 0.5987198352813721),
 ('екатеринбург', 0.5933181643486023),
 ('барнаул', 0.5915604829788208)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [10]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [11]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [65]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [49]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
pca = pca.fit(word_vectors)

word_vectors_pca = pca.transform(word_vectors)
means = word_vectors_pca.mean(axis=0)
std = word_vectors_pca.std(axis=0)

word_vectors_pca = (word_vectors_pca - means)/std

# and maybe MORE OF YOUR CODE here :)

In [50]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [27]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [52]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [24]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

tsne = TSNE(n_components = 2, perplexity=30)
word_tsne = tsne.fit_transform(word_vectors)

NameError: name 'word_vectors' is not defined

In [68]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1200', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [115]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    embedding_size = 100
    tokens = tokenizer.tokenize(phrase.lower())
    vectors = np.array([model.get_vector(token) for token in tokens if token in model.key_to_index])   
    
    vector = np.mean(vectors, axis=0)
    if vector.shape != (embedding_size,):
        return None
    return vector

In [15]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [19]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [22]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [25]:
from sklearn.manifold import TSNE
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [28]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1004', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [116]:
# compute vector embedding for all lines in data
data_vectors = [get_phrase_embedding(l) for l in data]
data_vectors = [l for l in data_vectors if l is not None]

In [117]:
def cos(vec1, vec2):
    return vec1.dot(vec2)/np.linalg.norm(vec1)/np.linalg.norm(vec2)

def find_nearest(query, k=10):    
    query_emebed = get_phrase_embedding(query)
    coss = [cos(data, query_emebed) for data in data_vectors]
    mask = np.argsort(coss)[-k:]
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    return np.array(data)[mask]

In [118]:
find_nearest('Im a gay')

array(['What does it mean when girls reply to a text a day after?\n',
       'What are the best interview questions to ask a singer songwriter?\n',
       'Is Lambton college in Toronto good to pursue masters?\n',
       'Has mathematics ever moved you to tears?\n', 'What is tcs tax?\n',
       'What are the differences between an Alaskan Husky and a Siberian Husky?\n',
       'What is the first letter of a five letter word meaning pace?\n',
       'It seems like life has no point for me?\n',
       'What Alan Rickman movie did you like the most?\n',
       'Which scholarships can an Indian student applying abroad for graduate studies apply for?\n'],
      dtype='<U1170')

In [119]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

Who wrote the Blue's Clues theme song?
Why do bookies still make money?
What are some good truth or dare questions to send via text?
How can one get a good college admission in Singapore?
What are the most interesting products and innovations that Federal Signal is coming out with in 2016?
How are the orders of a court conveyed to the concerned authorities?
Is canvas a protein fiber?
Which are the latest apps for Android mobiles?
Why are a lot of homes in India dirty and unhygienic? Is it just about income or also about a lack of desire to keep things clean?
What is headward erosion?



AssertionError: 

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.